In [ ]:
# %load 10_2022_load_config.py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
import seaborn as sns
import sys
import plotly.express as px
import plotly.io as pio
import yaml

sns.set_context("notebook", font_scale=1.4)
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)
plt.rcParams["figure.figsize"] = (16, 12)
plt.rcParams['savefig.dpi'] = 200
plt.rcParams['figure.autolayout'] = False
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['font.size'] = 16
plt.rcParams['lines.linewidth'] = 2.0
plt.rcParams['lines.markersize'] = 8
plt.rcParams['legend.fontsize'] = 14
pd.set_option('display.float_format', lambda x: '{:,.4f}'.format(x))


config_file = "10_2022_analysis.yaml"
with open(config_file) as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    configs = yaml.load(file, Loader=yaml.FullLoader)
    
# Run on server:
root = Path(configs['root'])
scratchDir = root/configs['scratchDir']
figuresDir = root/configs['figuresDir']
libraries = configs['libraries']


alphabetClrs = px.colors.qualitative.Alphabet
clrs = ["#f7ba65", "#bf4713", "#9c002f", "#d73d00", "#008080", "#004c4c"]
colors = {'grey': alphabetClrs[8], 
        'light_yellow': clrs[0],
        'darko': clrs[1],
        'maroon':clrs[2],
        'brighto': clrs[3],
        'teal':clrs[4],
        'darkteal':clrs[5]
       }

In [ ]:
from numpy.random import RandomState
def load_map(file, name):
    return (pd.read_csv(file)
            .assign(sample=name)[['barcode', 'number_of_reads', 'sample']]
            .drop_duplicates()
            .pivot(index="sample", columns='barcode', values='number_of_reads'))
    
    
def rarefaction(M, seed=0, depth=1000000):
    prng = RandomState(seed) # reproducible results
    noccur = np.sum(M, axis=1) # number of occurrences for each sample
    nvar = M.shape[1] # number of variables
    Mrarefied = np.empty_like(M)
    depths = []
    for i in range(M.shape[0]): # for each sample
        p = M.iloc[i] / float(noccur[i]) # relAb of each gene -> probability
        if depth >= noccur[i]:
            choice = prng.choice(nvar, noccur[i], p=p)
            depths.append(noccur[i])
        else:
            choice = prng.choice(nvar, depth, p=p)
            depths.append(depth)
        Mrarefied[i] = np.bincount(choice, minlength=nvar)
        
    return pd.DataFrame(Mrarefied, index=M.index, columns=M.columns), depths


def sat_curve(df, depths, cutoff=100):
    n_bcs = []
    n_reads = []
    for depth in depths:
        r, d = rarefaction(df, depth=depth)
        n_bcs.append((r>cutoff).sum(axis=1).iloc[0])
        n_reads.append(d[0])
    return pd.DataFrame([n_bcs, n_reads], index=['num_insertions', 'num_reads']).T


def depth_vs_num_inserts(pivot_map, depth, cutoffs):
    df_list = []
    for cutoff in cutoffs:
        df = sat_curve(pivot_map, depth, cutoff)
        df['ReadCutoff'] = f'{cutoff} Reads'
        df_list.append(df)
    return pd.concat(df_list)

In [ ]:
mapsL0Dir = Path(configs['mapsL0Dir'])
mapFiles = [mapsL0Dir/f"{lib}/{lib}.annotated.csv" for lib in libraries]

In [ ]:
nguyen_depth = sorted([250000, 500000, 750000] + list(range(50000, 9000000, 1000000)))
cutoffs = [5, 10, 50, 100, 500]

In [ ]:
curveDfs = []
for libFile in mapFiles:
    lib_map = load_map(libFile, libFile.stem)
    curves = depth_vs_num_inserts(lib_map, nguyen_depth, cutoffs).assign(library=libFile.stem)
    curveDfs.append(curves)
curveDf = pd.concat(curveDfs)

In [ ]:
curveDf['library'] = curveDf.library.str.split('.', expand=True, n=1)[0]

In [ ]:
fig = px.line(curveDf, 
              x="num_reads", 
              y="num_insertions", 
              color='ReadCutoff',
              color_discrete_map = {'5 Reads': colors['darkteal'], 
                                   '10 Reads': colors['teal'],
                                   '50 Reads': colors['maroon'],
                                   '100 Reads': colors['brighto'], 
                                   '500 Reads':colors['light_yellow']},
              labels = {'num_reads': "Sequencing Depth (millions of reads)",
                        'num_insertions': "Number of unique insertions",
                        'ReadCutoff': 'Detection Limit'}, 
              template = "plotly_white", 
              facet_col='library', facet_col_wrap=2,
              height=2000, 
              width=1000
             )
fig.update_traces(mode='markers+lines', line_width=4, marker_size=12)
fig.update_layout(
    
    font_size=14,
)

In [ ]:
pio.write_image(fig, figuresDir/'nguyen_sat_curves.png', width=1000, height=2000, scale=2)